<img alt="" src="images/tau-data-Indonesia.png" />
<center><h2><strong><font color="blue">Pendahuluan Basis Data untuk Data Science</font></strong></h2></center>
<center><h3><strong><font color="blue"><a href="https://tau-data.id/course/bdds/"> https://tau-data.id/course/bdds/ </a></font></strong></h3></center>

<img alt="" src="images/cover.jpg" />

<center><h2><strong><font color="blue">Pendahuluan Elasticsearch - 01</font></strong></h2></center>
<center><h3><strong><font color="blue"><a href="https://tau-data.id/lesson/pendahuluan-elasticsearch/">https://tau-data.id/lesson/pendahuluan-elasticsearch/</a></font></strong></h3></center>
<b><center><h3>(C) Taufik Sutanto</h3></center>

# Outline

1. Instalasi Java
2. Instalasi Elasticsearch
3. Configurasi Server Elasticsearch
4. Instalasi Head Plugin
5. Instalasi Python Modules
6. Index & Mapping di Elasticsearch
7. Importing Data
8. Basic queries

# Instalasi Java

1. Unduh java **JDK 8** + Install 
2. Atur **Path** ke JDK binary executable
3. **Test** instalasi
4. Beberapa catatan kecil instalasi Java.
 - **JAVA_HOME**
 - Linux **OpenJDK**
 


# Instalasi Elasticsearch

1. Unduh **Elasticsearch** + Extract ke root (optional) 
2. **Rename** folder jika diinginkan (optional) 
3. Install Elasticsearch sebagai service:
    - **C:\elasticsearch\bin\elasticsearch-service install**
4. Jalankan service: services.msc (di windows)
    - Rubah sebagai Automatic run di startup jika diinginkan
    - Tekan "Start"
5. Instalasi plugin **head** di browser Chrome

# instalasi python Module untuk Elastic search

* Buka terminal (command prompt) lalu jalankan perintah
* **pip install --upgrade elasticsearch**
* import module elasticsearch dengan cara
**import elasticsearch as es**

In [23]:
from elasticsearch import Elasticsearch as Es



In [24]:
server, port, timeout = 'localhost', 9200, 30 
# local host = 172.27.0.1
try:
    conEs = Es( [ {'host':server,'port':port,'timeout':timeout} ] )
    print('Connected to ElasticSearch, koneksi = "conEs"')
except:
    print('Error, tidak bisa terkoneksi ke ElasticSearch. Yakinkan server Es sudah berjalan dengan baik dan port serta ip server benar')

Connected to ElasticSearch, koneksi = "conEs"


In [25]:
# Berikutnya kita akan membuat index (~tabel di RDBMS) di Elastic
# pertama-tama kita perlu parameter-parameter berikut
idx = "news20"
nShards = 1
nReplicas = 0
similarity = "BM25"
# https://www.elastic.co/guide/en/elasticsearch/reference/current/index-modules-similarity.html

In [26]:
# Pertama tetapkan setting index
dSetting = { "settings": {"index":{"number_of_shards":nShards,"number_of_replicas":nReplicas} } }
# Yes!, ElasticSearch support Bahasa Indonesia
# https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-lang-analyzer.html
print('Done!')

Done!


In [27]:
# Lalu gunakan setting index di atas untuk membuat index baru di Elastic
#conEs.indices.delete(index=idx,ignore=[400, 404]) # Hapus index kalau sudah ada / Mulai dari awal
conEs.indices.create(index=idx,body=dSetting,ignore=400) # Buat Indexnya
# Coba refresh plugin Head untuk memeriksa apakah index telah dibuat oleh Elastic

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'news20'}

In [28]:
import pickle 

try:
    f = open('data/20_news_groups.pckl', 'rb')
    data = pickle.load(f)
    f.close()
except:
    categories = ['sci.med', 'talk.politics.misc',  'rec.autos'] # Silahkan modifikasi
    data = fetch_20newsgroups(categories=categories,remove=('headers', 'footers', 'quotes'))
    f = open('data/20_news_groups.pckl', 'wb') # Agar tidak selalu butuh koneksi internet setiap mau load data
    pickle.dump(data, f) # perhatikan harus ada folder "data"
    f.close()

In [31]:
kategori = [data.target_names[label] for label in data.target] 
documents = [doc for doc in data.data]
print("Total dokumen = {}".format(len(documents)))
print(set(kategori), len(kategori))
print(documents[0])

Total dokumen = 1653
{'sci.med', 'rec.autos', 'talk.politics.misc'} 1653
I want to get a car alarm and I am thinking about getting an Ungo Box.
    Does anyone out there have any knowledge or experience with any of
    these alarms?  How about price ranges for the different models?
    Are these good car alarms?  Please email me any responces.

                cak3@ns3.lehigh.edu


In [34]:
A = [1, 2, 3]
B = ['a', 'b', 'c']

for a,b in zip(A,B):
    print(a,b)

1 a
2 b
3 c


In [35]:
for doc, kat in zip(documents, kategori):
    D = { "document":doc, "kategori":kat }
    conEs.index(index="news20", body=D)

In [47]:
# Sampai sini sebenarnya kita sudah bisa melakukan pencarian, contoh:
qry = 'mazda mpv'
result = conEs.search(index=idx, body={"query": {"match": {'document':qry}}})['hits']['hits']
result[0]

{'_index': 'news20',
 '_type': '_doc',
 '_id': 'O3ODeHIBzENlLH17Lu0Z',
 '_score': 17.153393,
 '_source': {'document': 'I was in the great storm.....my Mazda MPV was damaged so bad they are \ngoing to replace the top, doors and hood. It is Black so they will repaint \nthe entire vehicle...estimated cost around $7000 and repair time approx. 3 \nto 4 weeks.\n',
  'kategori': 'rec.autos'}}

In [37]:
result.keys()

dict_keys(['took', 'timed_out', '_shards', 'hits'])

In [45]:
result['hits']['hits'][:3]

[{'_index': 'news20',
  '_type': '_doc',
  '_id': 'O3ODeHIBzENlLH17Lu0Z',
  '_score': 17.153393,
  '_source': {'document': 'I was in the great storm.....my Mazda MPV was damaged so bad they are \ngoing to replace the top, doors and hood. It is Black so they will repaint \nthe entire vehicle...estimated cost around $7000 and repair time approx. 3 \nto 4 weeks.\n',
   'kategori': 'rec.autos'}},
 {'_index': 'news20',
  '_type': '_doc',
  '_id': 'X3ODeHIBzENlLH17YvLR',
  '_score': 7.8345003,
  '_source': {'document': 'Hi Netters\n\tI want to know if 13500 (w/o tax) is a good deal for 1993 Mazda 626 DX\nHow is the performance review so far on Mazda 626. Is it a good buy?\nPlease reply to me as I don\'t read this group often.\n\nThanks In advance\nRavi\n--\nRavi Kiran Puvvala\t\t| "The purpose of education is not merely, \nravi@merlin.dev.cdx.mot.com\t|  the assimilation of facts but blow all \nMotorola Codex, Boston  MA \t|  the money" - Ravi Puvvala',
   'kategori': 'rec.autos'}},
 {'_inde

In [ ]:
res = hasil_query['hits']
type(res)

In [ ]:
res.keys()

In [ ]:
res = res

# End of Module

In [ ]:
import warnings; warnings.simplefilter('ignore')
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/S_f2qV2_U00?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')